In [1]:
from poisoning import *
import os
import subprocess

# 1. First poisoning

Our goal is to modify a memory dump by overwriting the "swapper\0" string and check the behaviour of Volatility3. Overwrite also other process names with the signature to check if Volatility 3 is able to distinguish them.


In [15]:
homedir = "/home/dadmin"
symboldir = "profiles"
volpypath = "/home/dadmin/Dev/Repo/Forensic/volatility3"
datapath = "/home/dadmin/Dev/Repo/Forensic/data"
dumppath = f"{datapath}/memory_dmp.lime"
dumppath_corrupt1 = f"{datapath}/corrupt1_memory_dmp.lime"
dumppath_corrupt2 = f"{datapath}/corrupt2_memory_dmp.lime"
dumppath_moded = f"{datapath}/memory_dmp_moded.lime"
original_path = os.getcwd()

The pslist() and pslist2() below are for two cases, pslist() is where volatility runs in: python vol.py [options] mode and pslist2() is where volatility runs like: vol [options]. please change pslist() to pslist2() if you installed volatility3 through the setup.py in the cloned repo or as a pip3 package. 

In [3]:
def run_pslist(custom_dumppath):
    try:
        os.chdir(volpypath)
        
        # whereweare = subprocess.run(["pwd"], capture_output=True, text=True)
        # print(f"whereweare = {whereweare.stdout}")
        # whereis_python = subprocess.run(["whereis", "python"], capture_output=True, text=True)
        # print(f"python version = {whereis_python.stdout}")

        # Run the Volatility 3 command
        cmd = ["python", "vol.py", "-s", symboldir, "-f", custom_dumppath, "linux.pslist.PsList"]
        print(f"cmd = {' '.join(cmd)}")
        subprocess.run(cmd, check=True)

    finally:
        os.chdir(original_path)

def run_pslist2(custom_dumppath):
    try:
        os.chdir(volpypath)
        
        # whereweare = subprocess.run(["pwd"], capture_output=True, text=True)
        # print(f"whereweare = {whereweare.stdout}")
        # whereis_python = subprocess.run(["whereis", "python"], capture_output=True, text=True)
        # print(f"python version = {whereis_python.stdout}")

        # Run the Volatility 3 command
        cmd = ["vol", "-s", symboldir, "-f", custom_dumppath, "linux.pslist.PsList"]
        print(f"cmd = {' '.join(cmd)}")
        subprocess.run(cmd, check=True)

    finally:
        os.chdir(original_path)

In [4]:
run_pslist(dumppath)

cmd = python vol.py -s profiles -f /home/dadmin/Dev/Repo/Forensic/data/memory_dmp.lime linux.pslist.PsList
Volatility 3 Framework 2.7.1



OFFSET (V)	PID	TID	PPID	COMM	File output

0x980b8023cd40	1	1	0	systemd	Disabled
0x980b802399c0	2	2	0	kthreadd	Disabled
0x980b80238000	3	3	2	rcu_gp	Disabled
0x980b8023b380	4	4	2	rcu_par_gp	Disabled
0x980b80258000	5	5	2	slub_flushwq	Disabled
0x980b8025b380	6	6	2	netns	Disabled
0x980b802599c0	8	8	2	kworker/0:0H	Disabled
0x980b802819c0	10	10	2	mm_percpu_wq	Disabled
0x980b80280000	11	11	2	rcu_tasks_kthre	Disabled
0x980b80283380	12	12	2	rcu_tasks_rude_	Disabled
0x980b8028cd40	13	13	2	rcu_tasks_trace	Disabled
0x980b802899c0	14	14	2	ksoftirqd/0	Disabled
0x980b80288000	15	15	2	rcu_preempt	Disabled
0x980b8028b380	16	16	2	migration/0	Disabled
0x980b80a599c0	18	18	2	cpuhp/0	Disabled
0x980b80a58000	19	19	2	cpuhp/1	Disabled
0x980b80a5b380	20	20	2	migration/1	Disabled
0x980b80a5cd40	21	21	2	ksoftirqd/1	Disabled
0x980b80a83380	23	23	2	kworker/1:0H	Disabled
0x980b80a84d40	24	24	2	cpuhp/2	Disabled
0x980b80a819c0	25	25	2	migration/2	Disabled
0x980b80aa19c0	26	26	2	ksoftirqd/2	Disabled
0x980b80aa3380	28	

## 1.1 Modification of the swapper signature

In [5]:
swapper_signature = rb"swapper(\/0|\x00\x00)\x00\x00\x00\x00\x00\x00"
new_signature = rb"swapLOL(\/0|\x00\x00)\x00\x00\x00\x00\x00\x00"

Before running the next cell, kindly chmod 666 and chown username:group all memory dmp files.

In [6]:
duplicate_dump(dumppath, dumppath_corrupt1)
modify_swapsign_in_dump(dumppath_corrupt1, swapper_signature, new_signature)

Dump file duplicated: /home/dadmin/Dev/Repo/Forensic/data/corrupt1_memory_dmp.lime
Found swapper signature at offset: 3162213920
Modified 1 occurrences of swapper signature.


In [7]:
run_pslist(dumppath_corrupt1)

cmd = python vol.py -s profiles -f /home/dadmin/Dev/Repo/Forensic/data/corrupt1_memory_dmp.lime linux.pslist.PsList
Volatility 3 Framework 2.7.1



OFFSET (V)	PID	TID	PPID	COMM	File output



after testing volatility3 it can't perform pslist as expected because it don't find the swapper anymore perfect !

## 1.2 Overwrite process names

### Search by signature

In [8]:
process_signature = rb"slub_flushwq"
# new_signature = rb"hello_you___"
new_signature = rb"swapper(\/0)"

In [9]:
duplicate_dump(dumppath, dumppath_corrupt2)

Dump file duplicated: /home/dadmin/Dev/Repo/Forensic/data/corrupt2_memory_dmp.lime


In [10]:
modify_swapsign_in_dump(dumppath_corrupt2, process_signature, new_signature)

Found swapper signature at offset: 3155220632
Found swapper signature at offset: 3222558000
Found swapper signature at offset: 3223151648
Modified 3 occurrences of swapper signature.


In [11]:
run_pslist(dumppath_corrupt2)

cmd = python vol.py -s profiles -f /home/dadmin/Dev/Repo/Forensic/data/corrupt2_memory_dmp.lime linux.pslist.PsList
Volatility 3 Framework 2.7.1



OFFSET (V)	PID	TID	PPID	COMM	File output

0x980b8023cd40	1	1	0	systemd	Disabled
0x980b802399c0	2	2	0	kthreadd	Disabled
0x980b80238000	3	3	2	rcu_gp	Disabled
0x980b8023b380	4	4	2	rcu_par_gp	Disabled
0x980b80258000	5	5	2	swapper(\/0)	Disabled
0x980b8025b380	6	6	2	netns	Disabled
0x980b802599c0	8	8	2	kworker/0:0H	Disabled
0x980b802819c0	10	10	2	mm_percpu_wq	Disabled
0x980b80280000	11	11	2	rcu_tasks_kthre	Disabled
0x980b80283380	12	12	2	rcu_tasks_rude_	Disabled
0x980b8028cd40	13	13	2	rcu_tasks_trace	Disabled
0x980b802899c0	14	14	2	ksoftirqd/0	Disabled
0x980b80288000	15	15	2	rcu_preempt	Disabled
0x980b8028b380	16	16	2	migration/0	Disabled
0x980b80a599c0	18	18	2	cpuhp/0	Disabled
0x980b80a58000	19	19	2	cpuhp/1	Disabled
0x980b80a5b380	20	20	2	migration/1	Disabled
0x980b80a5cd40	21	21	2	ksoftirqd/1	Disabled
0x980b80a83380	23	23	2	kworker/1:0H	Disabled
0x980b80a84d40	24	24	2	cpuhp/2	Disabled
0x980b80a819c0	25	25	2	migration/2	Disabled
0x980b80aa19c0	26	26	2	ksoftirqd/2	Disabled
0x980b80aa3380	28	

As you can see here PID 5 have been modifid

In [12]:
process_signature_2 = rb"migration/0"

modify_extension_in_dump(dumppath_corrupt2, process_signature_2, swapper_signature)

Modified occurrence at offset: 3223361440 with new data.
Modified 1 occurrences of the wanted signature.


In [13]:
run_pslist(dumppath_corrupt2)

cmd = python vol.py -s profiles -f /home/dadmin/Dev/Repo/Forensic/data/corrupt2_memory_dmp.lime linux.pslist.PsList
Volatility 3 Framework 2.7.1



OFFSET (V)	PID	TID	PPID	COMM	File output



# 2. Second poisoning


## 2.1 Test Volatility3 Stability on modified task_struct

write a simple kernel module that modifies the task_struct of swapper\0 and check the system stability. 

Done ✅

In [17]:
run_pslist(dumppath_moded)

cmd = python vol.py -s profiles -f /home/dadmin/Dev/Repo/Forensic/data/memory_dmp_moded.lime linux.pslist.PsList
Volatility 3 Framework 2.7.1



OFFSET (V)	PID	TID	PPID	COMM	File output

0x93664022cd40	1	1	0	systemd	Disabled
0x9366402299c0	2	2	0	kthreadd	Disabled
0x93664022b380	3	3	2	rcu_gp	Disabled
0x936640228000	4	4	2	rcu_par_gp	Disabled
0x936640250000	5	5	2	slub_flushwq	Disabled
0x936640254d40	6	6	2	netns	Disabled
0x936640253380	8	8	2	kworker/0:0H	Disabled
0x936640284d40	10	10	2	mm_percpu_wq	Disabled
0x9366402819c0	11	11	2	rcu_tasks_kthre	Disabled
0x936640283380	12	12	2	rcu_tasks_rude_	Disabled
0x9366402899c0	13	13	2	rcu_tasks_trace	Disabled
0x93664028b380	14	14	2	ksoftirqd/0	Disabled
0x936640288000	15	15	2	rcu_preempt	Disabled
0x93664028cd40	16	16	2	migration/0	Disabled
0x936640a4b380	18	18	2	cpuhp/0	Disabled
0x936640a48000	19	19	2	cpuhp/1	Disabled
0x936640a4cd40	20	20	2	migration/1	Disabled
0x936640a499c0	21	21	2	ksoftirqd/1	Disabled
0x936640a73380	23	23	2	kworker/1:0H	Disabled
0x936640a70000	24	24	2	cpuhp/2	Disabled
0x936640a74d40	25	25	2	migration/2	Disabled
0x936640a999c0	26	26	2	ksoftirqd/2	Disabled
0x936640a98000	28	

Nothing have change.